In [1]:
import pandas as pd
import numpy as np
import os
import pandas_gbq
from google.cloud import bigquery
import glob
import openpyxl
import re

In [2]:
os.chdir('G:\\\\Drives compartilhados\\\\República.org\\\\4. Equipes\\\\Dados e Comunicação\\\\DADOS\\\\415 - Repositório de Dados\\\\Repositório Local')


In [3]:
df18 = pd.read_excel('MUNIC//Base_MUNIC_2018.xlsx', sheet_name='Cultura', usecols=['Cod Municipio','MCUL01','MCUL03','MCUL04','MCUL05','MCUL06'])
df18

,Cod Municipio,MCUL01,MCUL03,MCUL04,MCUL05,MCUL06
0,1100015,Secretaria em conjunto com outras políticas se...,Masculino,36,Parda,Ensino médio (2º Grau) completo
1,1100023,Órgão da administração indireta,Masculino,43,Parda,Especialização
2,1100031,Secretaria em conjunto com outras políticas se...,Feminino,59,Parda,Especialização
3,1100049,Órgão da administração indireta,Masculino,64,Parda,Ensino superior completo
4,1100056,Secretaria em conjunto com outras políticas se...,Feminino,54,Branca,Especialização
...,...,...,...,...,...,...
5565,5222005,Secretaria em conjunto com outras políticas se...,Masculino,38,Branca,Ensino superior incompleto
5566,5222054,Secretaria em conjunto com outras políticas se...,Feminino,32,Branca,Ensino superior completo
5567,5222203,Não possui estrutura,-,-,-,-
5568,5222302,Secretaria em conjunto com outras políticas se...,Masculino,44,Parda,Especialização


In [4]:
df21 = pd.read_excel('MUNIC//Base_MUNIC_2021.xlsx', sheet_name='Cultura', usecols=['CodMun','Mcul01','Mcul03','Mcul04','Mcul05','Mcul06'])
df21

,CodMun,Mcul01,Mcul03,Mcul04,Mcul05,Mcul06
0,1100015,Secretaria em conjunto com outras políticas se...,Masculino,39,Parda,Ensino superior completo
1,1100023,Setor subordinado diretamente à chefia do Exe...,Feminino,24,Branca,Ensino superior completo
2,1100031,Secretaria em conjunto com outras políticas se...,Feminino,51,Parda,Ensino superior completo
3,1100049,Órgão da administração indireta,Masculino,30,Parda,Ensino superior completo
4,1100056,Secretaria em conjunto com outras políticas se...,Feminino,57,Parda,Especialização
...,...,...,...,...,...,...
5565,5222005,Setor subordinado a outra secretaria,Masculino,43,Parda,Ensino superior completo
5566,5222054,Secretaria em conjunto com outras políticas se...,Feminino,35,Branca,Especialização
5567,5222203,Não possui estrutura,-,-,-,-
5568,5222302,Secretaria em conjunto com outras políticas se...,Feminino,36,Parda,Especialização


In [5]:
uf = pd.read_excel('MUNIC//Base_MUNIC_2021.xlsx', sheet_name = 'Variáveis externas', usecols=[0,1,2,3])
uf

,CodMun,UF,Cod UF,Mun
0,1100015,RO,11,Alta Floresta DOeste
1,1100023,RO,11,Ariquemes
2,1100031,RO,11,Cabixi
3,1100049,RO,11,Cacoal
4,1100056,RO,11,Cerejeiras
...,...,...,...,...
5565,5222005,GO,52,Vianópolis
5566,5222054,GO,52,Vicentinópolis
5567,5222203,GO,52,Vila Boa
5568,5222302,GO,52,Vila Propício


In [6]:
df18= df18.rename(columns={'Cod Municipio':'id_municipio',
                        'MCUL01':'caracterizacao_orgao_gestor',
                        'MCUL03':'genero',
                        'MCUL04' :'idade',
                        'MCUL05':'cor_raca',
                        'MCUL06':'grau_instrucao'}) 

In [7]:
df21= df21.rename(columns={'CodMun':'id_municipio',
                        'Mcul01':'caracterizacao_orgao_gestor',
                        'Mcul03':'genero',
                        'Mcul04' :'idade',
                        'Mcul05':'cor_raca',
                        'Mcul06':'grau_instrucao'}) 

In [8]:
##adicionando a coluna de 'ano' em cada um dos dfs
df18['ano']=2018
df21['ano']=2021 

In [9]:
df = pd.concat([df18,df21]) #juntando os dataframes

In [10]:
x= uf.pivot_table(columns=('CodMun', 'UF','Cod UF', 'Mun'), aggfunc='size')


In [11]:
uf = pd.DataFrame(x).reset_index()[['CodMun', 'UF','Cod UF', 'Mun']]

In [12]:
df = df.merge(uf, right_on='CodMun',left_on='id_municipio') #adicionando siglas e nome das UFs e dos municipios


In [13]:
df['ano'].unique()

array([2018, 2021], dtype=int64)

In [14]:
df.columns

Index(['id_municipio', 'caracterizacao_orgao_gestor', 'genero', 'idade',
       'cor_raca', 'grau_instrucao', 'ano', 'CodMun', 'UF', 'Cod UF', 'Mun'],
      dtype='object')

In [15]:
df = df.drop(['CodMun'], axis=1)

In [16]:
df = df.rename(columns={'UF':'sigla_uf',
                        'Cod UF':'cod_uf',
                        'Mun':'nome_municipio'}) #padronizando as colunas

In [17]:
df

,id_municipio,caracterizacao_orgao_gestor,genero,idade,cor_raca,grau_instrucao,ano,sigla_uf,cod_uf,nome_municipio
0,1100015,Secretaria em conjunto com outras políticas se...,Masculino,36,Parda,Ensino médio (2º Grau) completo,2018,RO,11,Alta Floresta DOeste
1,1100015,Secretaria em conjunto com outras políticas se...,Masculino,39,Parda,Ensino superior completo,2021,RO,11,Alta Floresta DOeste
2,1100023,Órgão da administração indireta,Masculino,43,Parda,Especialização,2018,RO,11,Ariquemes
3,1100023,Setor subordinado diretamente à chefia do Exe...,Feminino,24,Branca,Ensino superior completo,2021,RO,11,Ariquemes
4,1100031,Secretaria em conjunto com outras políticas se...,Feminino,59,Parda,Especialização,2018,RO,11,Cabixi
...,...,...,...,...,...,...,...,...,...,...
11135,5222203,Não possui estrutura,-,-,-,-,2021,GO,52,Vila Boa
11136,5222302,Secretaria em conjunto com outras políticas se...,Masculino,44,Parda,Especialização,2018,GO,52,Vila Propício
11137,5222302,Secretaria em conjunto com outras políticas se...,Feminino,36,Parda,Especialização,2021,GO,52,Vila Propício
11138,5300108,Secretaria exclusiva,Masculino,64,Branca,Ensino médio (2º Grau) completo,2018,DF,53,Brasília


In [24]:
#padronizando os dados da coluna 'genero'
df['genero']=np.where(df['genero']=='(**)Sem gestor','Sem titular',df['genero'])
df['genero']=np.where(df['genero']=='(**) Sem gestor','Sem titular',df['genero'])
df['genero']=np.where(df['genero']=='Não informou','Sem dados',df['genero'])
df['genero']=np.where(df['genero']=='-','Sem dados',df['genero'])


#padronizando os dados da coluna 'idade'
df['idade']=np.where(df['idade']=='(*) Não soube informar',np.nan,df['idade'])
df['idade']=np.where(df['idade']=='(**)Sem gestor',np.nan,df['idade'])
df['idade']=np.where(df['idade']=='(**) Sem gestor',np.nan,df['idade'])
df['idade']=np.where(df['idade']=='Recusa',np.nan,df['idade'])
df['idade']=np.where(df['idade']=='Não informou',np.nan,df['idade'])
df['idade']=np.where(df['idade']=='-',np.nan,df['idade'])


#padronizando os dados da coluna 'cor_raca'

df['cor_raca']=np.where(df['cor_raca']=='(**) Sem gestor','Sem titular',df['cor_raca'])
df['cor_raca']=np.where(df['cor_raca']=='(**)Sem gestor','Sem titular',df['cor_raca'])
df['cor_raca']=np.where(df['cor_raca']=='-','Sem dados',df['cor_raca'])
df['cor_raca']=np.where(df['cor_raca']=='Não informou','Sem dados',df['cor_raca'])

#padronizando os dados da coluna 'escolaridade'

df['grau_instrucao']=np.where(df['grau_instrucao']=='(**)Sem gestor','Sem titular',df['grau_instrucao'])
df['grau_instrucao']=np.where(df['grau_instrucao']=='(**) Sem gestor','Sem titular',df['grau_instrucao'])
df['grau_instrucao']=np.where(df['grau_instrucao']=='Não informou','Sem dados',df['grau_instrucao'])
df['grau_instrucao']=np.where(df['grau_instrucao']=='-','Sem dados',df['grau_instrucao'])
df['grau_instrucao']=np.where(df['grau_instrucao']=='Ensino fundamental (1º Grau) incompleto','Até Ensino Fundamental',df['grau_instrucao'])
df['grau_instrucao']=np.where(df['grau_instrucao']=='Ensino fundamental (1º Grau) completo','Até Ensino Fundamental',df['grau_instrucao'])
df['grau_instrucao']=np.where(df['grau_instrucao']=='Ensino médio (2º Grau) incompleto','Até Ensino Médio',df['grau_instrucao'])
df['grau_instrucao']=np.where(df['grau_instrucao']=='Ensino médio (2º Grau) completo','Até Ensino Médio',df['grau_instrucao'])
df['grau_instrucao']=np.where(df['grau_instrucao']=='Ensino superior incompleto','Até Ensino Superior Completo',df['grau_instrucao'])
df['grau_instrucao']=np.where(df['grau_instrucao']=='Ensino superior completo','Até Ensino Superior Completo',df['grau_instrucao'])
df['grau_instrucao']=np.where(df['grau_instrucao']=='Especialização','Até Pós Graduação ou Mestrado',df['grau_instrucao'])
df['grau_instrucao']=np.where(df['grau_instrucao']=='Mestrado','Até Pós Graduação ou Mestrado',df['grau_instrucao'])
df['grau_instrucao']=np.where(df['grau_instrucao']=='Doutorado','Até Doutorado',df['grau_instrucao'])

In [29]:
df['caracterizacao_orgao_gestor']=df['caracterizacao_orgao_gestor'].str.title()

In [30]:
df['idade'] =pd.to_numeric(df['idade']) #transformando em numerico 


In [31]:
limites = [0, 30, 50,65,100]
categorias = ['Entre 18-29', 'Entre 30-49', 'Entre 50-64', 'Acima de 65']

df['faixa_etaria'] = pd.cut(df['idade'], bins=limites, labels=categorias)


In [32]:
df=df[['ano', 'sigla_uf','nome_municipio','id_municipio','caracterizacao_orgao_gestor', 'genero', 'cor_raca', 'grau_instrucao', 'faixa_etaria']] #reorganizando as colunas

Subindo para o GBQ

In [36]:
client = bigquery.Client()
dataset_ref = client.dataset('cargos_lideranca')

In [37]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 11140 entries, 0 to 11139
Data columns (total 9 columns):
 #   Column                       Non-Null Count  Dtype   
---  ------                       --------------  -----   
 0   ano                          11140 non-null  int64   
 1   sigla_uf                     11140 non-null  object  
 2   nome_municipio               11140 non-null  object  
 3   id_municipio                 11140 non-null  int64   
 4   caracterizacao_orgao_gestor  11140 non-null  object  
 5   genero                       11140 non-null  object  
 6   cor_raca                     11140 non-null  object  
 7   grau_instrucao               11140 non-null  object  
 8   faixa_etaria                 10721 non-null  category
dtypes: category(1), int64(2), object(6)
memory usage: 794.4+ KB


In [38]:
schema=[bigquery.SchemaField('ano','INTEGER',description='Ano de referência da observação'),
        bigquery.SchemaField('sigla_uf','STRING',description='Sigla da Unidade da Federação referente municipio'),
        bigquery.SchemaField('nome_municipio','STRING',description='Nome do município'),
        bigquery.SchemaField('id_municipio','INTEGER',description='Código do IBGE do município'),
        bigquery.SchemaField('caracterizacao_orgao_gestor','STRING',description='Caracterização do órgão no qual o gestor está'),
        bigquery.SchemaField('genero','STRING',description='Gênero autodeclarado ou não'),
        bigquery.SchemaField('cor_raca','STRING',description='Raça/cor da pessoa observada'),
        bigquery.SchemaField('grau_instrucao','STRING',description='Escolaridade da pessoa ou do vínculo observado com detalhamento na pós-graduação'),
        bigquery.SchemaField('faixa_etaria','STRING',description='faixa etária da observação')
        ]


In [39]:
table_ref = dataset_ref.table('MUNIC_perfil_gestor_cultura_tipo_orgao')
job_config = bigquery.LoadJobConfig(schema=schema)
job = client.load_table_from_dataframe(df,table_ref, job_config=job_config)
job.result() 

LoadJob<project=repositoriodedadosgpsp, location=US, id=c64f5069-7cdf-42b5-915d-36102f8d7429>